# User Guide

## Planning out the model

This document is intended to be a thought exercise and user guide on how to build models with the footings framework. Below are a series of items to think about as a model is planned out.

- What is the objective of the model?
- What should the model return?
- What steps can we break the model up into?
- What parameters will we need to pass to the model?
- Is there any metadata we want to attached to the model (e.g., run date/time, model version)?


Note these items are listed sequentially, though, there is likely to be some iteration between these items as users sit down to work on them.

**Objective of the model**

When looking to build a model, you first need to define what the objective of the model is. That is, what do we want the model to accomplish.

To demonstrate how to build a model with the footings framework we are going to use the example objective - **build a simple model to calculate disabled life reserves (DLR)**. A disabled life reserve is a reserve that is established once a policy holder has a qualifying event to receive future benefit payments. The reserve is the present value of expected future benefit payments with a decrement for lives inforce where the lives inforce is calculated using a claim termination rate (CTR).


**What do we want the model to return**

Knowing what we the model to do, one can start the planning process for what the model whould return.

Continuing with the DLR example, the model needs to return -

- output structured as a table with rows showing projected dates to the end of the benefit period,
- the expected future benefit over time (can change based on cost of living adjustments (COLA) or other offsets),
- the claim termination rate for each duration (i.e., each projected date),
- the lives inforce for each duration, and
- the discount factor for each duration (used to calcualte the present value of benefits).

The table would look similar to the table below.

+-----------------+-------------------+------+---------------+-----------------+----------------+
| Projected Dates | Expected Benefit  | CTR  | Lives Inforce | Discount Factor | PV of Benefits |
+=================+===================+======+===============+=================+================+
| 2019-12-31      | $0                | 0.00 | 1.000         | 1.000           |                |
+-----------------+-------------------+------+---------------+-----------------+----------------+
| 2020-01-31      | $100              | 0.05 | 0.950         | 0.980           |                |
+-----------------+-------------------+------+---------------+-----------------+----------------+
| 2020-02-28      | $100              | 0.06 | 0.892         | 0.960           |                |
+-----------------+-------------------+------+---------------+-----------------+----------------+
| ...             | ...               | ...  | ...           | ...             | ...            |
+-----------------+-------------------+------+---------------+-----------------+----------------+

In addition, attributes and keys such as policy id, claim id, etc. should  be added to the output to idenfiy who the reserve is calculated for.

**Steps to the model**

Having an understand of the objective of the model and what the model needs to produce, the model can be broken down into smaller components or steps.

Continuing with the example, we can use the bulleted list of items to return from the model from the section above to drive the steps.

Thus, the steps will be -

1. Create a table with projected durations (i.e., dates) given an as of date and the ending benefit date for a disabled claimant.
2. Calculate the expected future benefits considering COLA and any offsets.
3. Calculate the claim termination rate for each duration given the attributes that drive the claim termination rate (e.g., gender, attained age, etc.).
4. Calculate the lives inforce for each duration.
5. Calculate the discount rate for each duration given an interest rate.
6. Calculate the present value (pv) of future benefits.
7. Prepare table for final output.


**Parameters to the model**

Building off the objective, what the model needs to return, and the steps, one should have an idea of what parameters we need to expose to the end user to be able to use the model. Defining the parameters is probably the part that will require the most iteration as any change to required output or the underlying steps can change the parameters that are required of the model. In addition, there is additional effort by the developer to think about the data type of the parameters, whether a restricted set of values should be used, is there a min or max limit to values passed, etc.

Continuing with our example of building a DLR model -

- Step #1 requires an input for the `valuation date` (i.e., the as of date) and the `benefit end date`.
- Step #2 requires the `benefit amount` at the valuation date and a `COLA percent` (we will assume no offsets).
- Step #3 requires `gender` and `occupation class` as parameters as they drive the claim termination rate (this is assumed for demo only).
- Step #4 does not require any inputs as it will use the claim termination rate developed in step #3.
- Step #5 requires an `interest rate` to calcualte the discount.
- Step #6 does not require any inputs as we are calculating the final pv of benefits.
- Step #7 doe not require any inputs as it simply is preping the data for final output.

In addition, we want to include parameters for policy id and claim id so we known who the reserve is calcualted for.

Thus, the final list of parameters is below -

- policy_id
- claim_id
- valuation_date
- benefit_end_date
- benefit_amount
- cola_percent
- gender
- occupation_class
- interest_rate


**Meta data for the model**

Lastly, when developing a model with the footings framework, we need to decide what metadata we want the model to contain. Metadata is any data we want to exist within the model that does not need to be passed as a parameter by the model user. Examples of this include having the model generate a timestamp of the current time the model is run or the model version.

Continuing with the DLR example, we will add a metadata element to capture the run date/time.


## Building the model

A model in the footings framework is a standalone class which inherits from the built-in `Footing` class. The model needs to be defined using standard python notation of `class Model(Footing)`. In addition, the model needs to be decorated with `@model(steps=["step_1", "step_2", ...])` and include a list of the names of the steps to the model. Inheriting from Footing provides the key methods to run, audit, and visualize the model while decorating the model with `@model` defines the step order to run the model and provides some control checks. 

Steps in the model are represented as methods. It is a best practice to use an underscore when creating a step method. In addition, all steps are required to be decorated with `@step` and a listing of what attributes are used and assets or placeholders are impacted. 

Below is how our example DLR model is built using the Footings framework. Each component will be discussed in the following sections.

In [1]:
from footings import define_parameter, define_meta, define_asset, model, Footing, step
from footings.model_tools import run_date_time, create_frame
import pandas as pd

STEPS = [
    "_create_projected_frame", 
    "_calculate_expected_benefit", 
    "_calculate_ctr", 
    "_calculate_lives_inforce", 
    "_calculcate_discount", 
    "_calculate_pv_benefits", 
    "_prepare_final_output",
]

@model(steps=STEPS)
class DLRModel(Footing):
    """Simple DLR model example."""

    policy_id = define_parameter(
        dtype=str, 
        description="The policy id of the disabled policy holder."
    )
    claim_id = define_parameter(
        dtype=str, 
        description="The claim id of the disabled policy holder."
    )
    valuation_date = define_parameter(
        dtype=pd.Timestamp, 
        description="The valuation date the model needs to be ran as of."
    )
    benefit_end_date = define_parameter(
        dtype=pd.Timestamp, 
        description="The benefit end date for the disabled policy holder."
    )
    benefit_amount = define_parameter(
        dtype=float, 
        description="The benefit amount as of the valuation date."
    )
    cola_percent = define_parameter(
        dtype=float, 
        min_val=0, 
        max_val=0.05, 
        description="The cost of living adjustment."
    )
    gender = define_parameter(
        dtype=str, 
        allowed=["M", "F"], 
        description="The gender of the disabled policy holder."
    )
    occupation_class = define_parameter(
        dtype=str, 
        allowed=["1", "2"], 
        description="The occupation class of the disabled policy holder."
    )
    interest_rate = define_parameter(
        dtype=float, 
        min_val=0, 
        max_val=0.10, 
        description="The interest rate used for discounting."
    )
    run_date_time = define_meta(
        meta=run_date_time, 
        dtype=pd.Timestamp, 
        description="The run date/time the model is executed."
    )
    table = define_asset(
        dtype=pd.DataFrame, 
        description="The table produced with the projected reserve amount."
    )

    @step(uses=["valuation_date", "benefit_end_date"], impacts=["table"])
    def _create_projected_frame(self):
        """Create projected frame."""
        self.table = create_frame(
            start_dt=self.valuation_date, 
            end_dt=self.benefit_end_date, 
            frequency="Y", 
            col_date_nm="DATE", 
            duration_year="DURATION_YEAR"
        )

    @step(uses=["table", "benefit_amount", "cola_percent"], impacts=["table"])
    def _calculate_expected_benefit(self):
        """Calculate expected benefits."""
        self.table["BENEFIT_AMOUNT"] = (
            self.benefit_amount * (1 + self.cola_percent) ** self.table["DURATION_YEAR"]
        )

    @step(uses=["table", "gender", "occupation_class"], impacts=["table"])
    def _calculate_ctr(self):
        """Calculate claim termination rate."""
        ctrs = {
            ("M", "1"): 0.01,
            ("M", "2"): 0.02,
            ("F", "1"): 0.015,
            ("F", "2"): 0.025,
        }
        self.table["CTR"] = ctrs.get((self.gender, self.occupation_class))

    @step(uses=["table"], impacts=["table"])
    def _calculate_lives_inforce(self):
        """Calculate lives inforce at the end of the duration."""
        self.table["LIVES_INFORCE"] = (1 - self.table["CTR"]).cumprod()

    @step(uses=["table", "interest_rate"], impacts=["table"])
    def _calculcate_discount(self):
        """Calculate discount factor at the end of the duration."""
        self.table["DISCOUNT_FACTOR"] = (
            1 / (1 + self.interest_rate) ** self.table["DURATION_YEAR"]
        )

    @step(uses=["table"], impacts=["table"])
    def _calculate_pv_benefits(self):
        """Calcualte present value (PV) of benefits."""
        prod_cols = ["LIVES_INFORCE", "DISCOUNT_FACTOR", "BENEFIT_AMOUNT"]
        self.table["PV_FUTURE_BENEFITS"] = (
            self.table[prod_cols].prod(axis=1).iloc[::-1].cumsum()
        )

    @step(uses=["table", "policy_id", "claim_id", "run_date_time"], impacts=["table"])
    def _prepare_final_output(self):
        """Prepare table for final output.""" 
        self.table["POLICY_ID"] = self.policy_id
        self.table["CLAIM_ID"] = self.claim_id
        self.table["RUN_DATE_TIME"] = self.run_date_time
        col_order = [
            "RUN_DATE_TIME", 
            "POLICY_ID", 
            "CLAIM_ID", 
            "CTR", 
            "LIVES_INFORCE", 
            "DISCOUNT_FACTOR", 
            "BENEFIT_AMOUNT", 
            "PV_FUTURE_BENEFITS"
        ]
        self.table = self.table[col_order]


**Imports**

Building the model we start off importing a few libraries with the first set coming from the `footings` library.

- `define_parameter` is how a paramter is created in the Footings framework.
- `define_meta` is how metadata is added to a model in the Footings framework.
- `define_asset` is how an object that is created by the model is defined. In this example, we want the model to return a table, so we need to create a table under the model as an asset.
- `model` as mentiond earlier is the decorator that defines the step order of the model.
- `Footing` is the required parent class of a model.
- `step` is the decorator used to establish a step.

Next, from `footings.tools` two helper functions are imported - `run_date_time` and `create_frame` that are used by the model. Lastly, `pandas` is imported because it is used as the underlying object supporting the table. 



**Attributes**

Skipping `STEPS`, under `class DLRModel(Footing)` are the model attributes. Notice they are defined using `define_parameter`, `define_meta`, and `define_asset`. A parameter attribute will be exposed to the end user of the model. These attributes are frozen and cannot be changed once instantiation occurs. A meta attribute is not exposed to the end user as a model input. It is a frozen attribute that records the date/time when a model is instantiated. Finally, an asset parameter is an attribute that the model either creates or modifies. It is not exposed to the user as a parameter. However, on instantiation the attribute is created and can be modified by methods.

For all the attributes, we can define data types (dtype), descriptions, specifications on values, etc. Later on `help` is ran on the model and you will notice the descriptions and data types are actually added to the model documentation.


**Steps**

The steps to the model are ordered with a `list` and passed into `@model`. Notice that each step is a method under the class `DLRModel`. Each method is decorated using `@step` and displays what attributes the step uses and impacts. The model does nothing internally with these. It is instead used to help tell the story of what each step is doing. In addition, notice that each method does not return an object. Instead, each method modifies `self.table` which is the only asset the model produces and is returned once the model is finish running.

## Running the model

The subclassing `Footings` exposes the main methods - 

- `run` which executes the model returning all assets from the model. If more than one asset is created a tuple of results will be returned. In addition, `run` does take an opitional parameter `to_step` which will run the model to a specific step of the model and return itself.
- `audit` excecutest the model in audit mode. In audit mode, the model has a snapshot of itself taking at each step so each step can be analyzed independently of the other steps. This method is useful for satisfying any internal audit emulators describing the process used to create model output.
- `visualize` produces a visualization of the steps of the model as well as the interation among the different attributes.

These methods and a host of other information is available when running `help` on the object `DLRModel`.


In [2]:
help(DLRModel)

Help on class DLRModel in module __main__:

class DLRModel(footings.core.model.Footing)
 |  DLRModel(*, policy_id: 'str', claim_id: 'str', valuation_date: 'Timestamp', benefit_end_date: 'Timestamp', benefit_amount: 'float', cola_percent: 'float', gender: 'str', occupation_class: 'str', interest_rate: 'float') -> 'DLRModel'
 |  
 |  Simple DLR model example.
 |  
 |  Parameters
 |  ----------
 |  policy_id : str
 |      The policy id of the disabled policy holder.
 |  claim_id : str
 |      The claim id of the disabled policy holder.
 |  valuation_date : Timestamp
 |      The valuation date the model needs to be ran as of.
 |  benefit_end_date : Timestamp
 |      The benefit end date for the disabled policy holder.
 |  benefit_amount : float
 |      The benefit amount as of the valuation date.
 |  cola_percent : float
 |      The cost of living adjustment.
 |  gender : str
 |      The gender of the disabled policy holder.
 |  occupation_class : str
 |      The occupation class of the di

When viewing the documentation after running `help`, notice that the model returns a docstring with all the attributes split between parameters, meta, and assets with each assoiated data type and description. The Footings framework wants to make it easy to document models and turning code into readable documentation displays this concept.

To run the model, we first need to instantiate the model.

In [3]:
dlr_model = DLRModel(
    policy_id="M1",
    claim_id="C1",
    valuation_date=pd.Timestamp("2020-01-01"),
    benefit_end_date=pd.Timestamp("2030-12-31"),
    benefit_amount=100,
    cola_percent=0.02,
    gender="M",
    occupation_class="1",
    interest_rate=0.03,
)

The model can then be running calling the `run` method.

In [4]:
dlr_model.run()

,RUN_DATE_TIME,POLICY_ID,CLAIM_ID,CTR,LIVES_INFORCE,DISCOUNT_FACTOR,BENEFIT_AMOUNT,PV_FUTURE_BENEFITS
0,2020-11-13 05:40:38.189141,M1,C1,0.01,0.990000,0.970874,102.000000,1057.508396
1,2020-11-13 05:40:38.189141,M1,C1,0.01,0.980100,0.942596,104.040000,959.469561
2,2020-11-13 05:40:38.189141,M1,C1,0.01,0.970299,0.915142,106.120800,863.353429
3,2020-11-13 05:40:38.189141,M1,C1,0.01,0.960596,0.888487,108.243216,769.122294
4,2020-11-13 05:40:38.189141,M1,C1,0.01,0.950990,0.862609,110.408080,676.739186
5,2020-11-13 05:40:38.189141,M1,C1,0.01,0.941480,0.837484,112.616242,586.167864
6,2020-11-13 05:40:38.189141,M1,C1,0.01,0.932065,0.813092,114.868567,497.372795
7,2020-11-13 05:40:38.189141,M1,C1,0.01,0.922745,0.789409,117.165938,410.319143
8,2020-11-13 05:40:38.189141,M1,C1,0.01,0.913517,0.766417,119.509257,324.972758
9,2020-11-13 05:40:38.189141,M1,C1,0.01,0.904382,0.744094,121.899442,241.300156


To audit the model, call the `audit` method.

In [5]:
dlr_model.audit("dlr-model-audit.xlsx")

To visualize the model, call the `visualize` method. Note this is not fully implemented yet.

In [6]:
dlr_model.visualize()

To see more examples of the `footings` framework in action, visit the footings organizational [github site](https://www.github.com/footings).